# Relative Error of Numerical Difference

Note that "thymine" in this dataset should be changed to "cytosine", as stated in section 3 in Supporting Information.

In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_df_err(df, df_sub, ref=None):
    if ref is None:
        ref = df_sub
    # In case if column headers of reference is not the same to df
    if isinstance(df_sub, pd.DataFrame):
        df_sub.columns = df.columns
        ref.columns = df.columns
    df_err_val = df.sub(df_sub, axis="index")
    df_err_rel = df_err_val.div(ref, axis="index") * 100
    df_z = df - df.mean()
    ref_z = df_sub - df_sub.mean()
    df_err = {
        "MaxE/A^3": df_err_val.abs().max(),
        "MAD/A^3": df_err_val.abs().mean(),
        "RMSD/A^3": np.sqrt((df_err_val**2).mean()),
        "RelMaxE/%": df_err_rel.abs().max(),
        "RelMAD/%": df_err_rel.abs().mean(),
        "RelRMSD/%": np.sqrt((df_err_rel**2).mean()),
    }
    # In case df is pd.Series instead of pd.DataFrame 
    try:
        return pd.DataFrame(df_err).T
    except ValueError:
        return pd.Series(df_err)

## Read numerical isotropic polarizability

In [3]:
df_rijk = pd.read_csv("original_data/HR45-RIJK-NumChk.csv", header=[0, 1], index_col=[0])
df_rimp2 = pd.read_csv("original_data/HR45-RIMP2-NumChk.csv", header=[0, 1], index_col=[0])
df_corr2 = df_rimp2 - df_rijk

# change thymine to cytosine
df_num_index = list(df_rijk.index)
df_num_index[-1] = "cytosine"
df_rijk.index = df_num_index
df_rimp2.index = df_num_index
df_corr2.index = df_num_index

## Read analytical polarizability components, and transform to isotropic

In [4]:
df_anal = pd.read_csv("original_data/HR45-RIMP2-components.csv", header=[0, 1], index_col=[0, 1])
df_anal_rijk = 1 / 3 * (df_anal[("RI-JK", "xx")] + df_anal[("RI-JK", "yy")] + df_anal[("RI-JK", "zz")])
df_anal_rimp2 = 1 / 3 * (df_anal[("RI-MP2", "xx")] + df_anal[("RI-MP2", "yy")] + df_anal[("RI-MP2", "zz")])
df_anal_corr2 = df_anal_rimp2 - df_anal_rijk

## Analysis of Error Induced by Finite Difference (Table S8)

In [5]:
pd.DataFrame({
    "RI-MP2 loose": get_df_err(df_rimp2["aug-cc-pVTZ-loose"], df_anal_rimp2["aug-cc-pVTZ"]).loc["RelRMSD/%"],
    "RI-MP2 tight": get_df_err(df_rimp2["aug-cc-pVTZ-tight"], df_anal_rimp2["aug-cc-pVTZ"]).loc["RelRMSD/%"],
    "RI-JK loose": get_df_err(df_rijk["aug-cc-pVTZ-loose"], df_anal_rijk["aug-cc-pVTZ"]).loc["RelRMSD/%"],
    "RI-JK tight": get_df_err(df_rijk["aug-cc-pVTZ-tight"], df_anal_rijk["aug-cc-pVTZ"]).loc["RelRMSD/%"],
    "RI-Corr2 loose": get_df_err(df_corr2["aug-cc-pVTZ-loose"], df_anal_corr2["aug-cc-pVTZ"], df_anal_rimp2["aug-cc-pVTZ"]).loc["RelRMSD/%"],
    "RI-Corr2 tight": get_df_err(df_corr2["aug-cc-pVTZ-tight"], df_anal_corr2["aug-cc-pVTZ"], df_anal_rimp2["aug-cc-pVTZ"]).loc["RelRMSD/%"],
}).style.format("{:.3f}")

,RI-MP2 loose,RI-MP2 tight,RI-JK loose,RI-JK tight,RI-Corr2 loose,RI-Corr2 tight
5e-4,1.625,0.000,0.000,0.000,1.625,0.000
1e-3,0.395,0.002,0.001,0.001,0.395,0.001
2e-3,0.046,0.006,0.005,0.005,0.046,0.002
4e-3,0.031,0.025,0.019,0.019,0.020,0.008
8e-3,0.101,0.101,0.075,0.075,0.033,0.033
1.6e-2,0.415,0.415,0.309,0.309,0.138,0.138
